<a href="https://colab.research.google.com/github/Julita257/UMwF/blob/Case-2/Case_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Przegląd danych:

*   Podaj liczbę filmów Sci-Fi
*   Pokaż rozkład ocen komedii
*   Podaj średnią ocen wszystkich filmów akcji oraz 3 filmy najwyżej ocenianych????? ***(Dopytać się)***

2. System rekomendacyjny

*  Zbuduj system na podstawie algorytmu SVD oraz kNNwithMeans
*  Czym różni się algorytm kNN with means od standardowego kNN?
*  Wykorzystaj metodę hiperparametryzacji GridSearch do wyboru liczby sąsiadów od 2-6
*  W ocenie metod wykorzystaj walidację krzyżową
3. Podaj rekomendacje po obejrzeniu filmu: Jumanji  oraz Flint

1. Dane

In [1]:
import pandas as pd


In [2]:
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
all_scores = ratings['rating'].count()
all_scores

100836

In [6]:
movie_count = movies['movieId'].count()
movie_count

9742

In [7]:
user_count = len(ratings['userId'].unique())
user_count

610

In [8]:
AVG_score_user = all_scores / user_count
AVG_score_user

165.30491803278687

In [9]:
AVG_score_movie = all_scores / movie_count
AVG_score_movie

10.350646684459043

In [10]:
combined_data = pd.merge(ratings, movies, on='movieId')
combined_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


1. Przegląd danych

In [11]:
sci_fi_movies = combined_data[combined_data['genres'].str.contains('Sci-Fi')]
sci_fi_count = sci_fi_movies['movieId'].nunique()

In [12]:
print(f'Liczba filmów Sci-Fi: {sci_fi_count}')

Liczba filmów Sci-Fi: 980


In [13]:
comedy_movies = combined_data[combined_data['genres'].str.contains('Comedy')]
comedy_distribution = comedy_movies['rating'].value_counts(normalize=True).sort_index()

In [14]:
comedy_distribution

rating
0.5    0.016183
1.0    0.033723
1.5    0.022918
2.0    0.087189
2.5    0.064784
3.0    0.212685
3.5    0.130233
4.0    0.247331
4.5    0.071544
5.0    0.113410
Name: proportion, dtype: float64

In [15]:
action_movies = combined_data[combined_data['genres'].str.contains('Action')]
AVG_score_action = action_movies['rating'].mean()
AVG_score_action

3.447984331646809

In [25]:
average_ratings = action_movies.groupby(['movieId', 'title'])['rating'].mean().reset_index()
sorted_movies = average_ratings.sort_values(by='rating', ascending=False)
top_three_action_movies = sorted_movies.head(3)
top_three_action_movies

,movieId,title,rating
1239,72142,Love Exposure (Ai No Mukidashi) (2008),5.0
1692,138632,Tokyo Tribe (2014),5.0
1593,115727,Crippled Avengers (Can que) (Return of the 5 D...,5.0


2. System rekomendacyjny

In [16]:
! pip install scikit-surprise
from surprise import Dataset, Reader
from surprise import SVD, KNNWithMeans
from surprise.model_selection import cross_validate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162998 sha256=4adfc7987f6645f95cd543e2856099d2d37e2306889e5d86add0803d472b1bf5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [17]:
reader = Reader(rating_scale=(combined_data['rating'].min(), combined_data['rating'].max()))
data = Dataset.load_from_df(combined_data[['userId', 'movieId', 'rating']], reader)

In [18]:
svd = SVD()
SVD_result = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8764  0.8704  0.8677  0.8712  0.8832  0.8738  0.0055  
MAE (testset)     0.6710  0.6705  0.6693  0.6682  0.6747  0.6708  0.0022  
Fit time          1.73    2.87    2.82    2.53    1.74    2.34    0.51    
Test time         0.27    0.50    0.45    0.16    0.11    0.30    0.15    


In [19]:
knn_means = KNNWithMeans()
knn_result = cross_validate(knn_means, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9007  0.8994  0.8962  0.8927  0.8954  0.8969  0.0029  
MAE (testset)     0.6860  0.6866  0.6874  0.6819  0.6842  0.6852  0.0020  
Fit time          0.13    0.24    0.27    0.29    0.16    0.22    0.06    
Test time         2.56    2.94    3.50    1.69    2.09    2.56    0.63    


In [20]:
from surprise.model_selection import GridSearchCV

In [21]:
param_grid = {'k': [2, 3, 4, 5, 6], 'sim_options': {'name': ['pearson_baseline'], 'user_based': [True]}}
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=35)
gs.fit(data)
print(f"Najlepsze parametry: {gs.best_params['rmse']}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [22]:
print(f"Najlepsze parametry: {gs.best_params['rmse']}")

Najlepsze parametry: {'k': 6, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}}


In [27]:
best_knn_params = {
    'k': 6,
    'sim_options': {
        'name': 'pearson_baseline',
        'user_based': True  # user-based CF
    }
}

svd = SVD()
knn_with_means = KNNWithMeans(k=best_knn_params['k'], sim_options=best_knn_params['sim_options'])

Walidacja krzyżowa

In [28]:
svd_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
knn_results = cross_validate(knn_with_means, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8718  0.8736  0.8740  0.8679  0.8790  0.8733  0.0036  
MAE (testset)     0.6706  0.6714  0.6713  0.6658  0.6751  0.6708  0.0030  
Fit time          1.73    2.09    1.74    1.54    1.81    1.78    0.18    
Test time         0.44    0.21    0.26    0.12    0.11    0.23    0.12    
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing simil

In [29]:
print("Średni RMSE dla SVD: ", svd_results['test_rmse'].mean())
print("Średni MAE dla SVD: ", svd_results['test_mae'].mean())

print("Średni RMSE dla KNN with Means: ", knn_results['test_rmse'].mean())
print("Średni MAE dla KNN with Means: ", knn_results['test_mae'].mean())


Średni RMSE dla SVD:  0.8732560059491888
Średni MAE dla SVD:  0.6708302505695616
Średni RMSE dla KNN with Means:  0.9114978473698494
Średni MAE dla KNN with Means:  0.6918512297335657


In [30]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [38]:
import numpy as np
def generate_recommendations(movie_title, data, svd_model, top_n=3):
    movie_ids = data[data['title'].str.contains(movie_title, case=False, na=False)]['movieId'].unique()

    users_who_liked_movie = data[(data['movieId'].isin(movie_ids)) & (data['rating'] > data['rating'].mean())]['userId'].unique()

    recommendations = []
    for movie_id in data['movieId'].unique():
        if movie_id not in movie_ids:
            mean_rating = np.mean([svd_model.predict(user_id, movie_id).est for user_id in users_who_liked_movie])
            recommendations.append((movie_id, mean_rating))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:top_n]

    top_recommendations = [(data[data['movieId'] == movie_id]['title'].iloc[0], rating) for movie_id, rating in top_recommendations]

    return top_recommendations

# Wywołanie funkcji dla "Jumanji" i "Flint"
jumanji_recommendations = generate_recommendations("Jumanji", combined_data, svd)
flint_recommendations = generate_recommendations("Flint", combined_data, svd)

print("Rekomendacje po obejrzeniu 'Jumanji':")
for title, rating in jumanji_recommendations:
    print(f"{title}: {rating:.2f}")

print("\nRekomendacje po obejrzeniu 'Flint':")
for title, rating in flint_recommendations:
    print(f"{title}: {rating:.2f}")


Rekomendacje po obejrzeniu 'Jumanji':
Shawshank Redemption, The (1994): 4.51
Philadelphia Story, The (1940): 4.42
Streetcar Named Desire, A (1951): 4.41

Rekomendacje po obejrzeniu 'Flint':
Rear Window (1954): 4.52
Dallas Buyers Club (2013): 4.50
Seventh Seal, The (Sjunde inseglet, Det) (1957): 4.48
